In [7]:
import pandas as pd

In [8]:
data = pd.read_csv("../data/features/joggen_1-2025-04-27_17-01-13_features.csv")

In [11]:
data.head(5)

,Unnamed: 0,accelerometer_mean_x,accelerometer_std_x,accelerometer_mean_y,accelerometer_std_y,accelerometer_mean_z,accelerometer_std_z,accelerometer_sma,accelerometer_energy_x,accelerometer_domfreq_x,...,gyroscope_domfreq_x,magnetometer_mean_x,magnetometer_std_x,magnetometer_mean_y,magnetometer_std_y,magnetometer_mean_z,magnetometer_std_z,magnetometer_sma,magnetometer_energy_x,magnetometer_domfreq_x
0,2025-04-27 17:01:15.740,-1.826829,8.143798,2.310101,8.072506,0.470573,4.949983,16.377533,364984.026478,2,...,5,-25.819762,20.263831,8.462583,22.847264,-17.051547,13.296668,71.862128,8.720115e+06,0
1,2025-04-27 17:01:16.740,-2.580298,9.789006,1.707520,12.194701,0.519537,5.848314,22.693571,545731.396088,3,...,5,-31.500190,8.877316,26.156551,14.045326,-14.482577,10.166157,75.996363,1.031674e+07,0
2,2025-04-27 17:01:17.740,-3.372568,11.303422,2.540696,13.771349,0.430337,6.139754,26.879179,752591.121827,3,...,6,-29.746317,8.344365,30.945987,6.287361,-18.387213,4.789982,79.079517,9.196578e+06,0
3,2025-04-27 17:01:18.740,-2.048130,11.521227,2.774471,13.379070,-0.230299,6.555503,26.623908,705710.302118,3,...,6,-32.478820,7.300087,30.392958,6.214930,-15.036985,5.240493,77.908763,1.081520e+07,0
4,2025-04-27 17:01:19.740,-3.344913,12.750042,2.532596,14.391576,0.236367,7.353381,29.571517,924712.871236,3,...,6,-31.500601,8.602857,30.845768,6.527357,-14.685341,6.172191,77.046724,1.029292e+07,0


In [12]:
data.describe

<bound method NDFrame.describe of                   Unnamed: 0  accelerometer_mean_x  accelerometer_std_x  \
0    2025-04-27 17:01:15.740             -1.826829             8.143798   
1    2025-04-27 17:01:16.740             -2.580298             9.789006   
2    2025-04-27 17:01:17.740             -3.372568            11.303422   
3    2025-04-27 17:01:18.740             -2.048130            11.521227   
4    2025-04-27 17:01:19.740             -3.344913            12.750042   
..                       ...                   ...                  ...   
174  2025-04-27 17:04:09.740             -2.228914            10.409062   
175  2025-04-27 17:04:10.740             -0.174695             9.359451   
176  2025-04-27 17:04:11.740             -3.221361            10.057984   
177  2025-04-27 17:04:12.740             -2.434638            10.827003   
178  2025-04-27 17:04:13.740             -0.282219             9.733290   

     accelerometer_mean_y  accelerometer_std_y  accelerometer_mea

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179 entries, 0 to 178
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               179 non-null    object 
 1   accelerometer_mean_x     179 non-null    float64
 2   accelerometer_std_x      179 non-null    float64
 3   accelerometer_mean_y     179 non-null    float64
 4   accelerometer_std_y      179 non-null    float64
 5   accelerometer_mean_z     179 non-null    float64
 6   accelerometer_std_z      179 non-null    float64
 7   accelerometer_sma        179 non-null    float64
 8   accelerometer_energy_x   179 non-null    float64
 9   accelerometer_domfreq_x  179 non-null    int64  
 10  gyroscope_mean_x         179 non-null    float64
 11  gyroscope_std_x          179 non-null    float64
 12  gyroscope_mean_y         179 non-null    float64
 13  gyroscope_std_y          179 non-null    float64
 14  gyroscope_mean_z         1

In [16]:
df_test = pd.read_parquet("../data/raw_data_db_cache.parquet")

In [19]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2344703 entries, 0 to 2344702
Data columns (total 30 columns):
 #   Column             Dtype              
---  ------             -----              
 0   result             object             
 1   table              int64              
 2   _start             datetime64[ns, UTC]
 3   _stop              datetime64[ns, UTC]
 4   _time              datetime64[ns, UTC]
 5   _measurement       object             
 6   app_version        object             
 7   device_id          object             
 8   device_name        object             
 9   file_hash          object             
 10  label              object             
 11  measurement_group  object             
 12  platform           object             
 13  recording_time     object             
 14  accelerometer_x    float64            
 15  accelerometer_y    float64            
 16  accelerometer_z    float64            
 17  gravity_x          float64            
 18  gr